In [1]:
case class State[S, +A](runS : S => (A, S)) {
    def map[B](f: A => B): State[S, B] = {
        State[S, B]((s) => {
            val (a, s1) = runS(s)
            (f(a), s1)
        })
    }
    
    def flatMap[B](f: A => State[S, B]):State[S, B] = {
        State[S, B](s => {
            val (a, s1) = runS(s)
            f(a) runS s1
        })
    }
}

defined class State

In [2]:
def getState[S]:State[S, S] = State[S, S](s => s -> s)
def setState[S](s:S):State[S, Unit] = State(_ => ((), s))
def pureState[S, A](a: A):State[S, A] = State[S, A](s => (a, s))

defined function getState
defined function setState
defined function pureState

In [3]:
// FoldLeft is tail recursive. 
import scala.annotation.tailrec
def foldLeft[A, B](xs:List[A], init:B, f:(B, A) => B):B = {
    @tailrec
    def go(result:B, rem: List[A]):B = {
        rem match {
            case Nil => result
            case x :: xs => go(f(result, x), xs)
        }
    }
    go(init, xs)
    //xs.foldLeft(init)(f)
}

import scala.annotation.tailrec

defined function foldLeft
defined function zipIndex

In [4]:
zipIndex(new Range(0, 10000, 1).toList) // StackOverflow

: 

### Tail call elimination

- self-recursive calls replaced with a a single jump instruction => tail recursion as a loop
- JVM stores the 

In [5]:
// Limitations - Mutual recursion 

object MutualRecursion {
// @tailrec
def even [ A ]( ns : List [ A ]): Boolean =
    ns match {
        case Nil => true
        case x :: xs => odd ( xs )
    }

def odd [A ]( ns : List [A ]): Boolean =
    ns match {
        case Nil => false
        case x :: xs => even ( xs )
    }
}

defined object MutualRecursion

In [6]:
MutualRecursion.even(new Range(0, 10000, 1).toList)

: 

## A technique to avoid stack overflow
In a trampolined program, instead of each
step calling the next, functions yield the next step to a
single control loop known as the trampoline. 

Operational Monad - Turn any call into a tail call that can be subsequently eliminated

Trampolined programs - because of its yielding nature - model for cooperative coroutines

Trampoline monad to a free monad

## Trampolines
Trampoline - computation that can be stepped through
- One of two steps - More or Done

In [7]:
object TrampolineVanilla {
    sealed trait Trampoline[+A] {
        import Trampoline._
        @tailrec
        final def runT: A = 
          this match {
              case More(k) => k().runT
              case Done(v) => v
          }
    }

    object Trampoline{
        case class More[+A](k: () => Trampoline[A]) extends Trampoline[A]
        case class Done[+A](result: A) extends Trampoline[A]
    }
}

defined object TrampolineVanilla

In [8]:
object MutualRecursionTrampoline {
    import TrampolineVanilla.Trampoline
    import Trampoline._
    def even [ A ]( ns : List [ A ]): Trampoline[Boolean] =
        ns match {
            case Nil => Done(true)
            case x :: xs => More(() => odd ( xs ))
        }

    def odd [A ]( ns : List [A ]): Trampoline[Boolean] =
        ns match {
            case Nil => Done(false)
            case x :: xs => More(() => even ( xs ))
        }
}

defined object MutualRecursionTrampoline

In [9]:
MutualRecursionTrampoline.even(new Range(0, 10000, 1).toList)

res8: TrampolineVanilla.Trampoline[Boolean] = More(
  ammonite.$sess.cmd7$Helper$MutualRecursionTrampoline$$$Lambda$2710/1705102322@1e1dc2
)

In [10]:
res8.runT

res9: Boolean = true

### Trampolines on State monad example

In [11]:
object StateTrampoline {
    import TrampolineVanilla._
    import Trampoline._
    case class State[S, +A](runS : S => Trampoline[(A, S)]) {
        def map[B](f: A => B): State[S, B] = {
            State[S, B]((s) => {
                val (a, s1) = runS(s).runT // Running a computation. If you aren't running at the tail 
                Done((f(a), s1)) // Why Done?
            })
        }

        def flatMap[B](f: A => State[S, B]):State[S, B] = {
            State[S, B](s => {
                val (a, s1) = runS(s).runT // Not in a tail call. Can't be wrapped in Trampoline
                More(() => f(a) runS s1) // Why More? and Not Done?
            })
        }
    }
    def pureState[S, A](a: A):State[S, A] = State[S, A](s => Done((a, s)))
    def getState[S]:State[S, S] = State[S, S](s => Done(s -> s))
    def setState[S](s:S):State[S, Unit] = State(_ => Done(((), s)))
    
    def zipIndex[A](as: List[A]):List[(Int, A)] = {
        val initial = pureState[Int, List[(Int, A)]](List.empty[(Int, A)])
        foldLeft[A, State[Int, List[(Int, A)]]](as, initial, (acc, a) => for {
            xs <- acc
            n <- getState
            _ <- setState(n+1)
        } yield (n, a) :: xs).runS(0).runT._1.reverse
    }
}

defined object StateTrampoline

In [12]:
zipIndex(new Range(0, 10000, 1).toList) // StackOverflow

: 

# Trampoline as a Monad
- Trampoline with Flatmap

In [13]:
sealed trait TrampolineF[+A] {
    import TrampolineF._
    @tailrec
    final def runT: A = 
      this match {
          case MoreF(k) => k().runT
          case DoneF(v) => v
      }
    
    def flatMap[B](f: A => TrampolineF[B]) = MoreF(() => f(runT))
}

object TrampolineF{
    case class MoreF[+A](k: () => TrampolineF[A]) extends TrampolineF[A]
    case class DoneF[+A](result: A) extends TrampolineF[A]
}

defined trait TrampolineF
defined object TrampolineF

In [14]:
object StateTrampolinePlus {
    import TrampolineF._
    case class State[S, +A](runS : S => TrampolineF[(A, S)]) {
        def map[B](f: A => B): State[S, B] = {
            State[S, B]((s) => {
                val (a, s1) = runS(s).runT // Running a computation. If you aren't running at the tail 
                DoneF((f(a), s1)) // Why Done?
            })
        }

        def flatMap[B](f: A => State[S, B]):State[S, B] = {
            State[S, B](s => MoreF(() => runS(s) flatMap {
                case (a, s1) => MoreF(() => f(a) runS s1)
            }))
        }
    }
}

defined object StateTrampolinePlus

In [22]:
object TrampolineWithFlatMap {
    sealed trait Trampoline[+A] {
        import Trampoline._
        @tailrec
        final def runT: A = 
          resume match {
              case Right(a) => a
              case Left(k) => k().runT
          }

        def map[B](f: A => B):Trampoline[B] = {
          flatMap[B]((a) => Done(f(a)))
        }
        
        def flatMap[B](f: A => Trampoline [B]): Trampoline [B] =
            this match {
                case FlatMap (a , g) =>
                    FlatMap(a, (x: Any) => g (x) flatMap f)
                case x => FlatMap(x , f)
            }
                
         def resume: Either[() => Trampoline[A], A] = this match {
            case Done(v) => Right(v)
            case More(k) => Left(k)
            case FlatMap(a, f) => a match {
                case Done(v) => f(v).resume
                case More(k) => Left(() => k() flatMap f)
                case FlatMap(b, g) => b.flatMap((x: Any) => g(x) flatMap f).resume
            } 
        }
    }

    object Trampoline {
        case class More[+A](k: () => Trampoline[A]) extends Trampoline[A]
        case class Done[+A](result: A) extends Trampoline[A]
        private case class FlatMap[A, +B](sub: Trampoline[A], k: A => Trampoline[B]) extends Trampoline[B]
    }
}

defined object TrampolineWithFlatMap

## Examples

In [23]:
object Example {
    import TrampolineWithFlatMap._
    import Trampoline._
    def fib(n: Int):Trampoline[Int] = {
        if(n <= 1) Done(n) else {
            for {
                x <- More(() => fib(n - 1))
                y <- More(() => fib(n - 2))
            } yield x + y
        }
    }
}

defined object Example

In [33]:
Example.fib(40).runT

res32: Int = 102334155

In [58]:
object CooperativeMultitasking {
    import TrampolineWithFlatMap._
    import Trampoline._
    def zip [A, B](a:Trampoline[A], b: Trampoline [B]): Trampoline [( A ,B )] =
    (a.resume , b.resume ) match {
        case ( Right (a), Right (b)) =>
            Done (( a , b ))
        case ( Left (a) , Left (b )) =>
            More (() => zip(a(), b()))
        case ( Left (a) , Right (b )) =>
            More (() => zip(a(), Done(b)))
        case ( Right (a), Left (b)) =>
            More (() => zip(Done(a), b()))
    }
    
    def hello(id:String) : Trampoline [Unit] = for {
        _ <- Done(print(s"${id}-!!!!Hello!!!!!"))
        _ <- Done(println(s"${id}-World !"))
    } yield ()
}

defined object CooperativeMultitasking

In [61]:
(CooperativeMultitasking.zip (CooperativeMultitasking.hello("1"), CooperativeMultitasking.hello("2"))).runT

1-!!!!Hello!!!!!2-!!!!Hello!!!!!1-World !
2-World !


res60: (Unit, Unit) = ((), ())

# Coroutines for Cooperative scheduling

Coroutines are computer program components that generalize subroutines for non-preemptive multitasking, by allowing execution to be suspended and resumed.

Trampolines are coroutines(allow suspension and resume) that allow cooperative scheduling using `Function0` type constructor

In [63]:
// Abstract over the type constructor
sealed trait Functor[F[+_]]{
    def map[A, B](fa: F[A])(f: A => B):F[B]
}

object FreeEx {
    import FreeEx.Free._
    sealed trait Free[S[+_], +A] {
        private case class FlatMap[S[+_], A, +B](a: Free[S, A], f: A => Free[S, B]) extends Free[S, A]
        
        def flatMap[B](f: A => Free[S, B]): Free[S, B] =
            this match {
            case FlatMap (a, g) =>
                FlatMap[S,A,B](a, (x) => g(x) flatMap f)
            case x => FlatMap[S, A, B](x, f)
        }
        
        final def resume(implicit S: Functor[S]):Either[S[Free[S, A]], A] = {
            this match {
                case Done(a) => Right(a)
                case More(k) => Left(k)
                case FlatMap(a, f) => a match {
                    case Done(a) => f(a).resume
                    case More(k) => Left(S.map(k)(_ flatMap f)) // S[Free[S, B]]
                    case FlatMap(b, g) => b.flatMap((x:Any) => g(x) flatMap f).resume //Free[S, A]
                }
            }
        }
    }
    
    object Free {
        case class Done[S[+_], +A](a: A) extends Free[S, A]
        case class More[S[+_], +A](k: S[Free[S, A]]) extends Free[S, A]
    }
    
    type Trampoline[+A] = Free[Function0, A]
}

cmd63.sc:13: type mismatch;
 found   : A => ammonite.$sess.cmd63.Helper.FreeEx.Free[S,B]
 required: Any => ammonite.$sess.cmd63.Helper.FreeEx.Free[S,B]
                FlatMap[S,A,B](a, (x) => g(x) flatMap f)
                                                      ^cmd63.sc:13: type mismatch;
 found   : Free.this.FlatMap[S,A,B]
 required: ammonite.$sess.cmd63.Helper.FreeEx.Free[S,B]
                FlatMap[S,A,B](a, (x) => g(x) flatMap f)
                              ^cmd63.sc:14: type mismatch;
 found   : Free.this.FlatMap[S,A,B]
 required: ammonite.$sess.cmd63.Helper.FreeEx.Free[S,B]
            case x => FlatMap[S, A, B](x, f)
                                      ^cmd63.sc:22: type mismatch;
 found   : Either[S[ammonite.$sess.cmd63.Helper.FreeEx.Free[S,Any]],Any]
 required: Either[S[ammonite.$sess.cmd63.Helper.FreeEx.Free[S,A]],A]
                    case Done(a) => f(a).resume
                                         ^cmd63.sc:23: type mismatch;
 found   : A => ammonite.$sess.cmd63

: 